[https://docs.arbor-sim.org/en/stable/tutorial/single_cell_detailed_recipe.html](https://docs.arbor-sim.org/en/stable/tutorial/single_cell_detailed_recipe.html)

In [1]:
import arbor
from arbor import mpoint
from arbor import mnpos

In [2]:
# Define the morphology by manually building a segment tree
tree = arbor.segment_tree()

# Start with segment 0: a cylindrical soma with tag 1
tree.append(mnpos, mpoint(0.0, 0.0, 0.0, 2.0), mpoint( 40.0, 0.0, 0.0, 2.0), tag=1)
# Construct the first section of the dendritic tree with tag 3,
# comprised of segments 1 and 2, attached to soma segment 0.
tree.append(0,     mpoint(40.0, 0.0, 0.0, 0.8), mpoint( 80.0,  0.0, 0.0, 0.8), tag=3)
tree.append(1,     mpoint(80.0, 0.0, 0.0, 0.8), mpoint(120.0, -5.0, 0.0, 0.8), tag=3)
# Construct the rest of the dendritic tree: segments 3, 4 and 5.
tree.append(2,     mpoint(120.0, -5.0, 0.0, 0.8), mpoint(200.0,  40.0, 0.0, 0.4), tag=3)
tree.append(3,     mpoint(200.0, 40.0, 0.0, 0.4), mpoint(260.0,  60.0, 0.0, 0.2), tag=3)
tree.append(2,     mpoint(120.0, -5.0, 0.0, 0.5), mpoint(190.0, -30.0, 0.0, 0.5), tag=3)
# Construct a special region of the tree made of segments 6, 7, and 8
# differentiated from the rest of the tree using tag 4.
tree.append(5,     mpoint(190.0, -30.0, 0.0, 0.5), mpoint(240.0, -70.0, 0.0, 0.2), tag=4)
tree.append(5,     mpoint(190.0, -30.0, 0.0, 0.5), mpoint(230.0, -10.0, 0.0, 0.2), tag=4)
tree.append(7,     mpoint(230.0, -10.0, 0.0, 0.2), mpoint(360.0, -20.0, 0.0, 0.2), tag=4)
# Construct segments 9 and 10 that make up the axon with tag 2.
# Segment 9 is at the root, where its proximal end will be connected to the
# proximal end of the soma segment.
tree.append(mnpos, mpoint( 0.0, 0.0, 0.0, 2.0), mpoint(  -70.0, 0.0, 0.0, 0.4), tag=2)
tree.append(9,     mpoint(-70.0, 0.0, 0.0, 0.4), mpoint(-100.0, 0.0, 0.0, 0.4), tag=2)

morph = arbor.morphology(tree);

In [3]:
# (2) Create and populate the label dictionary.
labels = arbor.label_dict()

# Regions:

# Add labels for tag 1, 2, 3, 4
labels['soma'] = '(tag 1)'
labels['axon'] = '(tag 2)'
labels['dend'] = '(tag 3)'
labels['last'] = '(tag 4)'

# Add a label for a region that includes the whole morphology
labels['all'] = '(all)'
# Add a label for the parts of the morphology with radius greater than 1.5 μm.
labels['gt_1.5'] = '(radius-ge (region "all") 1.5)'

# Join regions "last" and "gt_1.5"
labels['custom'] = '(join (region "last") (region "gt_1.5"))'

# Add a labels for the root of the morphology and all the terminal points
labels['root']     = '(root)'
labels['terminal'] = '(terminal)'

# Add a label for the terminal locations in the "custom" region:
labels['custom_terminal'] = '(restrict (locset "terminal") (region "custom"))'
# Add a label for the terminal locations in the "axon" region:
labels['axon_terminal'] = '(restrict (locset "terminal") (region "axon"))'

In [4]:
# (3) Create and populate the decor.
decor = arbor.decor()

# Set the default properties of the cell (this overrides the model defaults).
decor.set_property(Vm =-55)
decor.set_ion('na', int_con=10,   ext_con=140, rev_pot=50, method='nernst/na')
decor.set_ion('k',  int_con=54.4, ext_con=2.5, rev_pot=-77)

In [5]:
# Override the cell defaults.
decor.paint('"custom"', tempK=270)
decor.paint('"soma"',   Vm=-50)

In [6]:
# Paint density mechanisms.
decor.paint('"all"', arbor.density('pas'))
decor.paint('"custom"', arbor.density('hh'))
decor.paint('"dend"',  arbor.density('Ih', {'gbar': 0.001}))

In [7]:
# Place stimuli and spike detectors.
decor.place('"root"', arbor.iclamp(10, 1, current=2), 'iclamp0')
decor.place('"root"', arbor.iclamp(30, 1, current=2), 'iclamp1')
decor.place('"root"', arbor.iclamp(50, 1, current=2), 'iclamp2')
decor.place('"axon_terminal"', arbor.spike_detector(-10), 'detector')

In [8]:
# Single CV for the "soma" region
soma_policy = arbor.cv_policy_single('"soma"')
# Single CV for the "soma" region
dflt_policy = arbor.cv_policy_max_extent(1.0)
# default policy everywhere except the soma
policy = dflt_policy | soma_policy
# Set cv_policy
decor.discretization(policy)

In [9]:
# (4) Create the cell.
cell = arbor.cable_cell(morph, labels, decor)

In [10]:
# (5) Declare a probe.
probe = arbor.cable_probe_membrane_voltage('"custom_terminal"')

In [11]:
# (6) Create a class that inherits from arbor.recipe
class single_recipe (arbor.recipe):

    # (6.1) Define the class constructor
    def __init__(self, cell, probes):
        # The base C++ class constructor must be called first, to ensure that
        # all memory in the C++ class is initialized correctly.
        arbor.recipe.__init__(self)
        self.the_cell = cell
        self.the_probes = probes

        self.the_props = arbor.cable_global_properties()
        self.the_props.set_property(Vm=-65, tempK=300, rL=35.4, cm=0.01)
        self.the_props.set_ion(ion='na', int_con=10,   ext_con=140, rev_pot=50, method='nernst/na')
        self.the_props.set_ion(ion='k',  int_con=54.4, ext_con=2.5, rev_pot=-77)
        self.the_props.set_ion(ion='ca', int_con=5e-5, ext_con=2, rev_pot=132.5)
        self.the_props.catalogue.extend(arbor.allen_catalogue(), "")

    # (6.2) Override the num_cells method
    def num_cells(self):
        return 1

    # (6.3) Override the num_targets method
    def cell_kind(self, gid):
        return arbor.cell_kind.cable

    # (6.4) Override the cell_description method
    def cell_description(self, gid):
        return self.the_cell

    # (6.5) Override the probes method
    def probes(self, gid):
        return self.the_probes

    # (6.6) Override the connections_on method
    def connections_on(self, gid):
        return []

    # (6.7) Override the gap_junction_on method
    def gap_junction_on(self, gid):
        return []

    # (6.8) Override the event_generators method
    def event_generators(self, gid):
        return []

    # (6.9) Overrode the global_properties method
    def global_properties(self, gid):
        return self.the_props

In [13]:
# (7) Instantiate recipe
# Pass the probe in a list because that it what single_recipe expects.
recipe = single_recipe(cell, [probe])

In [14]:
# (8) Create an execution context
context = arbor.context()

In [16]:
# (9) Create a domain decomposition
domains = arbor.partition_load_balance(recipe, context)

In [17]:
# (10) Create a simulation
sim = arbor.simulation(recipe, domains, context)

In [19]:
# (11) Instruct the simulation to record the spikes and sample the probe
sim.record(arbor.spike_recording.all)

probe_id = arbor.cell_member(0,0)
handle = sim.sample(probe_id, arbor.regular_schedule(0.02))

In [20]:
# (12) Print or display the results
spikes = sim.spikes()
print(len(spikes), 'spikes recorded:')
for s in spikes:
    print(s)

0 spikes recorded:
